## BME 544 Assignment 4

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import os
import pandas as pd
import csv

FS = 1000
LEARNING_PERIOD = 5.0
MOVING_AVG_LEN_S = 0.15 # [s]
DIRECTORY = "A4_DC_2/"
BLACKLIST = ["A4_DC_2/208956_Clean.txt", "A4_DC_2/208956_NoFilter.txt", "A4_DC_2/208956_EMG.txt", "A4_DC_2/X1JQII_EMG.txt", "A4_DC_2/X1JQII_NoFilter.txt"]
PEAKS_CSV_PATH = "integrated_peaks.csv"

def normalize_signal(signal_data):
    """Normalize signal to range [0, 1] for plotting"""
    if len(signal_data) == 0:
        return []
    
    min_val = np.min(signal_data)
    max_val = np.max(signal_data)
    
    # Avoid division by zero
    if max_val == min_val:
        return np.zeros_like(signal_data)
    
    return (signal_data - min_val) / (max_val - min_val)

class FIRFilter:
    def __init__(self, order=200, low_freq=7, high_freq=15):
        nyq = 0.5 * FS
        low_normalized = low_freq / nyq
        high_normalized = high_freq / nyq
        
        self.coefficients = signal.firwin(order + 1, [low_normalized, high_normalized], pass_zero=False, window='hamming')
        self.buffer = np.zeros(len(self.coefficients))
    
    def process_sample(self, sample):
        self.buffer = np.roll(self.buffer, 1)
        self.buffer[0] = sample
        return np.dot(self.coefficients, self.buffer)
    
class FivePointDerivative:
    def __init__(self):
        self.buffer = np.zeros(5)
    
    def process_sample(self, sample):
        self.buffer = np.roll(self.buffer, -1)
        self.buffer[-1] = sample
        
        if np.count_nonzero(self.buffer) < 5:
            return 0  # Not enough samples yet
        
        return (-self.buffer[0] + 8*self.buffer[1] - 8*self.buffer[3] + self.buffer[4]) / (8 / FS)

class MovingAverageFilter:
    def __init__(self):
        window_size_n = int(MOVING_AVG_LEN_S * FS)
        self.window_size = window_size_n
        self.buffer = np.zeros(window_size_n)
    
    def process_sample(self, sample):
        self.buffer = np.roll(self.buffer, -1)
        self.buffer[-1] = sample
        return np.mean(self.buffer)
    
class ECGStreamSimulator:
    def __init__(self, file_path):
        self.file_path = file_path
        self.data = None
        self.time = None
        self.current_idx = 0

        df = pd.read_csv(self.file_path, sep='\t', skiprows=1, names=["Time", "Sig"])
        
        nonzero_indices = df["Sig"].to_numpy().nonzero()[0]  # Get indices of nonzero values
        if len(nonzero_indices) > 0:
            first_nonzero_idx = nonzero_indices[0]  # First nonzero value index
            df = df.iloc[first_nonzero_idx:]  # Keep data from first nonzero index onward

        self.time = df.iloc[:, 0].values
        self.data = df.iloc[:, 1].values
        self.max_idx = len(self.data) - 1
        
    def read_next_sample(self):
        if self.current_idx >= len(self.data):
            return None
        
        time_val = self.time[self.current_idx]
        signal_val = self.data[self.current_idx]
        self.current_idx += 1
        
        return (time_val, signal_val)
    
class ImprovedAdaptiveThresholdQRSDetector:
    def __init__(self, filepath):
        self.filepath = filepath
        self.bandpass_filter = FIRFilter()
        self.differentiator = FivePointDerivative()
        self.moving_avg = MovingAverageFilter()
        
        self.learning_samples = int(LEARNING_PERIOD * FS)
        self.is_learning = True
        self.learning_counter = 0
        
        self.filtered_signal_level_estimate = 0
        self.filtered_noise_level_estimate = 0
        self.integrated_signal_level_estimate = 0
        self.integrated_noise_level_estimate = 0

        self.Fthreshold1 = 0
        self.Fthreshold2 = 0
        self.Ithreshold1 = 0
        self.Ithreshold2 = 0
        
        self.peak_finding_window_width = 101
        
        self.Frr_buffer = []
        self.Frr_buffer_mean = 0
        self.Irr_buffer = []
        self.Irr_buffer_mean = 0
        
        self.filtered_peaks_x = []
        self.integrated_peaks_x = []
        self.Fthreshold1_arr = []
        self.Fthreshold2_arr = []
        self.Ithreshold1_arr = []
        self.Ithreshold2_arr = []
        
        self.time = []
        self.raw_signal = []
        self.filtered_signal = []
        self.derivative_signal = []
        self.squared_signal = []
        self.integrated_signal = []

        self.fallback_max = 0
        self.max_hr = 110 # [bpm]
    
    
    def log_peaks_to_csv(self, peaks):
        """
        Log peaks to a CSV file, appending each file's peaks as a new row
        
        Args:
            filepath (str): Path of the input file
            peaks (list): List of peak indices
        """
        # Prepare a row with filepath and peaks
        row = [self.filepath, str([int(peak) for peak in peaks])]
        
        # Append to CSV file
        with open(PEAKS_CSV_PATH, 'a', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(row)
        
        print(f"Peaks for {self.filepath} logged to {PEAKS_CSV_PATH}")

    def non_casual_learning(self):
        """Use scipy.signal.find_peaks to analyze learning data and set thresholds"""
        Ipeaks, Ipeak_properties = signal.find_peaks(
            self.integrated_signal, 
            distance=int(60 * FS / self.max_hr),  # Minimum 200ms between peaks
            height=np.percentile(self.integrated_signal, 50)
        )
        
        if len(Ipeaks) > 0:
            Ipeak_heights = Ipeak_properties['peak_heights']
            
            if len(Ipeak_heights) > 4:  # Need enough Ipeaks for meaningful statistics
                q1 = np.percentile(Ipeak_heights, 25)
                q3 = np.percentile(Ipeak_heights, 75)
                iqr = q3 - q1
                upper_limit = q3 + 1.5 * iqr
                lower_limit = q1 - 1.5 * iqr
                
                # Keep only Ipeaks within IQR bounds
                valid_Ipeaks_mask = (Ipeak_heights >= lower_limit) & (Ipeak_heights <= upper_limit)
                valid_Ipeak_heights = Ipeak_heights[valid_Ipeaks_mask]
                valid_Ipeaks = Ipeaks[valid_Ipeaks_mask]
                self.integrated_peaks_x.extend(valid_Ipeaks)
                self.Irr_buffer.extend(valid_Ipeaks[-8:])

                if len(valid_Ipeak_heights) > 0:
                    self.integrated_signal_level_estimate = np.mean(valid_Ipeak_heights)
                else: # NOTE: this most likely won't be reached
                    self.integrated_signal_level_estimate = np.mean(Ipeak_heights)
            else:
                self.integrated_signal_level_estimate = np.mean(Ipeak_heights)
            
            # Set noise level
            self.integrated_noise_level_estimate = 0.1 * self.integrated_signal_level_estimate
            
            # Calculate thresholds
            self.Ithreshold1 = self.integrated_noise_level_estimate + 0.25 * (self.integrated_signal_level_estimate - self.integrated_noise_level_estimate)
            self.Ithreshold2 = 0.5 * self.Ithreshold1
            
            print(f"Learning completed - Signal Level: {self.integrated_signal_level_estimate:.2f}, Noise Level: {self.integrated_noise_level_estimate:.2f}")
            print(f"Initial Thresholds - Threshold1: {self.Ithreshold1:.2f}, Threshold2: {self.Ithreshold2:.2f}")
        else:
            print("Learning failed - no peaks detected in learning phase of integrated signal")

            self.Ithreshold1 = 0.5
            self.Ithreshold2 = 0.25
        ###############################################################################################################################################################
        Fpeaks, Fpeak_properties = signal.find_peaks(
            self.filtered_signal, 
            distance=int(60 * FS / self.max_hr),
            height=np.percentile(self.filtered_signal, 50)
        )
        
        if len(Fpeaks) > 0:
            Fpeak_heights = Fpeak_properties['peak_heights']
            
            if len(Fpeak_heights) > 4:  # Need enough Fpeaks for meaningful statistics
                q1 = np.percentile(Fpeak_heights, 25)
                q3 = np.percentile(Fpeak_heights, 75)
                iqr = q3 - q1
                upper_limit = q3 + 1 * iqr
                lower_limit = q1 - 1 * iqr
                
                # Keep only Fpeaks within IQR bounds
                valid_Fpeaks_mask = (Fpeak_heights >= lower_limit) & (Fpeak_heights <= upper_limit)
                valid_Fpeak_heights = Fpeak_heights[valid_Fpeaks_mask]
                valid_Fpeaks = Fpeaks[valid_Fpeaks_mask]
                self.filtered_peaks_x.extend(valid_Fpeaks)
                self.Frr_buffer.extend(valid_Fpeaks[-8:])

                if len(valid_Fpeak_heights) > 0:
                    self.filtered_signal_level_estimate = np.mean(valid_Fpeak_heights)
                else: # NOTE: this most likely won't be reached
                    self.filtered_signal_level_estimate = np.mean(Fpeak_heights)
            else:
                self.filtered_signal_level_estimate = np.mean(Fpeak_heights)
            
            self.filtered_noise_level_estimate = 0.1 * self.filtered_signal_level_estimate
            
            self.Fthreshold1 = self.filtered_noise_level_estimate + 0.25 * (self.filtered_signal_level_estimate - self.filtered_noise_level_estimate)
            self.Fthreshold2 = 0.5 * self.Fthreshold1
            
            print(f"Learning completed - Signal Level: {self.filtered_signal_level_estimate:.2f}, Noise Level: {self.filtered_noise_level_estimate:.2f}")
            print(f"Initial Thresholds - Threshold1: {self.Fthreshold1:.2f}, Threshold2: {self.Fthreshold2:.2f}")
        else:
            print("Learning failed - no peaks detected in learning phase of integrated signal")
            self.Fthreshold1 = 0.5
            self.Fthreshold2 = 0.25
        
        self.is_learning = False
    
    def update_thresholds(self, peak_height, is_signal, is_filtered=False):
        if is_filtered:
            if is_signal:
                self.filtered_signal_level_estimate = 0.125 * peak_height + 0.75 * self.filtered_signal_level_estimate
            else:
                self.filtered_noise_level_estimate = 0.125 * peak_height + 0.75 * self.filtered_noise_level_estimate
            
            self.Fthreshold1 = self.filtered_noise_level_estimate + 0.25 * (self.filtered_signal_level_estimate - self.filtered_noise_level_estimate)
            self.Fthreshold2 = 0.5 * self.Fthreshold1
        else:
            if is_signal:
                self.integrated_signal_level_estimate = 0.125 * peak_height + 0.75 * self.integrated_signal_level_estimate
            else:
                self.integrated_noise_level_estimate = 0.125 * peak_height + 0.875 * self.integrated_noise_level_estimate
            
            self.Ithreshold1 = self.integrated_noise_level_estimate + 0.25 * (self.integrated_signal_level_estimate - self.integrated_noise_level_estimate)
            self.Ithreshold2 = 0.5 * self.Ithreshold1
    
    def is_potential_peak(self, signal, sample_idx=-1):
        if sample_idx == -1:
            pass
        else:
            half_window = self.peak_finding_window_width // 2
            
            if sample_idx < half_window:
                return False
            
            start = sample_idx - self.peak_finding_window_width + 1
            end = sample_idx + 1
            
            return signal[sample_idx - half_window] == max(signal[start:end])
    
    def process_sample(self, time_val, signal_val, sample_idx):
        filtered_val = self.bandpass_filter.process_sample(signal_val)
        derivative_val = self.differentiator.process_sample(filtered_val)
        squared_val = derivative_val ** 2
        integrated_val = self.moving_avg.process_sample(squared_val)
        
        self.time.append(time_val)
        self.raw_signal.append(signal_val)
        self.filtered_signal.append(filtered_val)
        self.derivative_signal.append(derivative_val)
        self.squared_signal.append(squared_val)
        self.integrated_signal.append(integrated_val)
        
        if self.is_learning:
            self.Fthreshold1_arr.append(0)
            self.Fthreshold2_arr.append(0)
            self.Ithreshold1_arr.append(0)
            self.Ithreshold2_arr.append(0)
        else:
            self.Fthreshold1_arr.append(self.Fthreshold1)
            self.Fthreshold2_arr.append(self.Fthreshold2)
            self.Ithreshold1_arr.append(self.Ithreshold1)
            self.Ithreshold2_arr.append(self.Ithreshold2)
        
        if self.is_learning:
            self.learning_counter += 1
            if self.learning_counter >= self.learning_samples: self.non_casual_learning()
        
        else:
            integrated_signal_potential_peak = self.is_potential_peak(self.integrated_signal, sample_idx)
            if integrated_signal_potential_peak:
                peak_val = self.integrated_signal[sample_idx - self.peak_finding_window_width // 2]
                
                # Signal detection based on Integrated signal threshold
                if peak_val > self.Ithreshold1:
                    self.update_thresholds(peak_val, is_signal=True)
                    current_rr = sample_idx - self.peak_finding_window_width // 2 - self.integrated_peaks_x[-1]
                    self.integrated_peaks_x.append(sample_idx - self.peak_finding_window_width // 2)
                    if current_rr > 0.8 * self.Irr_buffer_mean and current_rr < 1.2 * self.Irr_buffer_mean:
                        self.Irr_buffer = self.Irr_buffer[1:] if len(self.Irr_buffer) == 8 else self.Irr_buffer
                        self.Irr_buffer.append(current_rr)
                        self.Irr_buffer_mean = sum(self.Irr_buffer) / len(self.Irr_buffer)
                # Noise detection based on second threshold
                elif peak_val > self.Ithreshold2:
                    self.update_thresholds(peak_val, is_signal=False)


            elif sample_idx - self.integrated_peaks_x[-1] > self.Irr_buffer_mean * 1.66:
                search_back_start = max(0, int(sample_idx - self.Irr_buffer_mean * 1.66))
                search_back_end = sample_idx
                
                # Find max peak in search back window within threshold range
                search_window = self.integrated_signal[search_back_start:search_back_end]
                candidate_peaks = [
                    idx for idx, val in enumerate(search_window, start=search_back_start) 
                    if self.Ithreshold2 < val < self.Ithreshold1
                ]
                
                if candidate_peaks:
                    # Find the maximum peak value in candidates
                    missed_peak_idx = max(candidate_peaks, key=lambda idx: self.integrated_signal[idx])
                    
                    # Add missed peak
                    self.integrated_peaks_x.append(missed_peak_idx)
                    missed_peak_val = self.integrated_signal[missed_peak_idx]
                    self.update_thresholds(missed_peak_val, is_signal=True)

            # Check potential peaks in filtered signal
            filtered_signal_potential_peak = self.is_potential_peak(self.filtered_signal, sample_idx)
            if filtered_signal_potential_peak:
                peak_val = self.filtered_signal[sample_idx - self.peak_finding_window_width // 2]
                
                # Signal detection based on Filtered signal threshold
                if peak_val > self.Fthreshold1:
                    self.update_thresholds(peak_val, is_signal=True, is_filtered=True)
                    current_rr = sample_idx - self.peak_finding_window_width // 2 - self.filtered_peaks_x[-1]
                    self.filtered_peaks_x.append(sample_idx - self.peak_finding_window_width // 2)
                    if current_rr > 0.8 * self.Frr_buffer_mean and current_rr < 1.2 * self.Frr_buffer_mean:
                        self.Frr_buffer = self.Frr_buffer[1:] if len(self.Frr_buffer) == 8 else self.Frr_buffer
                        self.Frr_buffer.append(current_rr)
                        self.Frr_buffer_mean = sum(self.Frr_buffer) / len(self.Frr_buffer)
                
                # Noise detection based on second threshold
                elif peak_val > self.Fthreshold2:
                    self.update_thresholds(peak_val, is_signal=False, is_filtered=True)

            elif sample_idx - self.filtered_peaks_x[-1] > self.Frr_buffer_mean * 1.66:
                search_back_start = max(0, int(sample_idx - self.Frr_buffer_mean * 1.66))
                search_back_end = sample_idx
                
                # Find max peak in search back window within threshold range
                search_window = self.filtered_signal[search_back_start:search_back_end]
                candidate_peaks = [
                    idx for idx, val in enumerate(search_window, start=search_back_start) 
                    if self.Fthreshold2 < val < self.Fthreshold1
                ]
                
                if candidate_peaks:
                    # Find the maximum peak value in candidates
                    missed_peak_idx = max(candidate_peaks, key=lambda idx: self.filtered_signal[idx])
                    
                    # Add missed peak
                    self.filtered_peaks_x.append(missed_peak_idx)
                    missed_peak_val = self.filtered_signal[missed_peak_idx]
                    self.update_thresholds(missed_peak_val, is_signal=True)
        return

    def plot_signal_analysis(self):
        initial_transients_list = ["A4_DC_2/XR50E7_Clean.txt",
            "A4_DC_2/M19ZSJ_NoFilter.txt",
            "A4_DC_2/V1QPTY_NoFilter.txt",
            "A4_DC_2/XR50E7_NoFilter.txt",
            "A4_DC_2/IM9878_Clean.txt",
            "A4_DC_2/CR41NG_EMG.txt",
            "A4_DC_2/ASJFJR_Clean.txt",
            "A4_DC_2/ME18T9_NoFilter.txt",
            "A4_DC_2/8752D3_Clean.txt",
            "A4_DC_2/5F3D6G_EMG.txt",
            "A4_DC_2/LM5SAG_NoFilter.txt",
            "A4_DC_2/LM5SAG_EMG.txt",
            "A4_DC_2/POTATO_Clean.txt",
            "A4_DC_2/CR41NG_NoFilter.txt",
            "A4_DC_2/LWJ3G7_NoFilter.txt",
            "A4_DC_2/C5N560_EMG.txt",
            "A4_DC_2/FWEIFD_NoFilter.txt",
            "A4_DC_2/C5N560_No_Filter.txt",
            "A4_DC_2/TH781G_EMG.txt",
            "A4_DC_2/V1QPTY_Clean.txt",
            "A4_DC_2/XR50E7_NoFilter.txt",
            "A4_DC_2/IM9878_Clean.txt",
            "A4_DC_2/CR41NG_EMG.txt",
            "A4_DC_2/ASJFJR_Clean.txt",
            "A4_DC_2/ME18T9_NoFilter.txt",
            "A4_DC_2/LM5SAG_NoFilter.txt",
            "A4_DC_2/LM5SAG_EMG.txt",
            "A4_DC_2/CR41NG_NoFilter.txt",
            "A4_DC_2/LWJ3G7_NoFilter.txt",
            "A4_DC_2/C5N560_EMG.txt",
            "A4_DC_2/FWEIFD_NoFilter.txt",
            "A4_DC_2/C5N560_No_Filter.txt",
            "A4_DC_2/TH781G_EMG.txt",
            "A4_DC_2/LM5SAG_Clean.txt",
            "A4_DC_2/LWJ3G7_EMG.txt",
            "A4_DC_2/WXC872_Clean.txt",
            "A4_DC_2/NHJSK9_NoFilter.txt",
            "A4_DC_2/WXC872_EMG.txt",
            "A4_DC_2/IM9878_NoFilter.txt",
            "A4_DC_2/C83JD0_Clean.txt",
            "A4_DC_2/C5N560_Clean.txt",
            "A4_DC_2/CR41NG_Clean.txt",
            "A4_DC_2/WXC872_NoFilter.txt"
        ]
        import matplotlib.pyplot as plt
        
        # Exclude first 1000 samples if the file is in the transients list
        start_idx = 1000 if self.filepath in initial_transients_list else 0

        # Create figure with subplots
        fig, axs = plt.subplots(5, 1, figsize=(15, 20), sharex=True)

        # Plot Raw Signal
        axs[0].plot(self.time[start_idx:], self.raw_signal[start_idx:] / np.max(self.raw_signal[start_idx:]), label='Raw Signal', color='gray')
        axs[0].set_title('Raw ECG Signal')
        axs[0].legend()

        # Plot Filtered Signal
        axs[1].plot(self.time[start_idx:], self.filtered_signal[start_idx:] / np.max(self.filtered_signal[start_idx:]), label='Filtered Signal', color='blue')
        axs[1].set_title('Filtered Signal with Thresholds')
        axs[1].legend()

        # Plot Derivative Signal
        axs[2].plot(self.time[start_idx:], self.derivative_signal[start_idx:] / np.max(self.derivative_signal[start_idx:]), label='Derivative Signal', color='purple')
        axs[2].set_title('Derivative Signal')
        axs[2].legend()

        # Plot Squared Signal
        axs[3].plot(self.time[start_idx:], self.squared_signal[start_idx:] / np.max(self.squared_signal[start_idx:]), label='Squared Signal', color='brown')
        axs[3].set_title('Squared Signal')
        axs[3].legend()

        # Plot Integrated Signal
        axs[4].plot(self.time[start_idx:], self.integrated_signal[start_idx:] / np.max(self.integrated_signal[start_idx:]), label='Integrated Signal', color='green')
        axs[4].scatter(
            [self.time[idx] - 0.2 for idx in self.integrated_peaks_x if idx >= start_idx],
            [self.integrated_signal[idx] / np.max(self.integrated_signal[start_idx:]) for idx in self.integrated_peaks_x if idx >= start_idx],
            color='red', label='Integrated Peaks'
        )
        axs[4].plot(self.time[start_idx:], self.raw_signal[start_idx:] / np.max(self.raw_signal[start_idx:]), label='Raw Signal', color='gray')
        axs[4].plot(self.time[start_idx:], self.Ithreshold1_arr[start_idx:] / np.max(self.integrated_signal[start_idx:]), label='Integrated Threshold 1', color='blue', linestyle='--')
        axs[4].plot(self.time[start_idx:], self.Ithreshold2_arr[start_idx:] / np.max(self.integrated_signal[start_idx:]), label='Integrated Threshold 2', color='orange', linestyle='--')
        axs[4].set_title('Integrated Signal with Thresholds')
        axs[4].legend()

        plt.tight_layout()
        plt.show()
        self.log_peaks_to_csv(self.integrated_peaks_x)

# Prepare CSV file with header before processing
def prepare_peaks_csv():
    with open(PEAKS_CSV_PATH, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Filepath', 'Peaks'])

prepare_peaks_csv()
with os.scandir(DIRECTORY) as entries:
    for entry in entries:
        if entry.is_file() and entry.path not in BLACKLIST:
            print(f"Processing {entry.path}...")
            ecg_stream = ECGStreamSimulator(entry.path)
            qrs_detector = ImprovedAdaptiveThresholdQRSDetector(entry.path)
            
            sample_idx = 0
            while True:
                sample = ecg_stream.read_next_sample()
                if sample is None:
                    break
                    
                time_val, signal_val = sample
                qrs_detector.process_sample(time_val, signal_val, sample_idx)
                sample_idx += 1

            qrs_detector.plot_signal_analysis()